In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def prepare_data():
    """Load and preprocess breast cancer dataset."""
    # Load dataset
    data = load_breast_cancer()
    X, y = data.data, data.target
    
    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1234
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Convert to PyTorch tensors
    X_train = torch.from_numpy(X_train.astype(np.float32))
    X_test = torch.from_numpy(X_test.astype(np.float32))
    y_train = torch.from_numpy(y_train.astype(np.float32)).view(-1, 1)
    y_test = torch.from_numpy(y_test.astype(np.float32)).view(-1, 1)
    
    return X_train, X_test, y_train, y_test

class BinaryClassifier(nn.Module):
    """Simple neural network for binary classification."""
    def __init__(self, input_features):
        super(BinaryClassifier, self).__init__()
        self.linear = nn.Linear(input_features, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

def train_model(model, X_train, y_train, num_epochs=100, learning_rate=0.01):
    """Train the model with given parameters."""
    criterion = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        # Forward pass
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Log progress
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

def evaluate_model(model, X_test, y_test):
    """Evaluate model performance on test set."""
    with torch.no_grad():
        y_pred = model(X_test)
        y_pred_classes = y_pred.round()
        accuracy = y_pred_classes.eq(y_test).sum() / float(y_test.shape[0])
        return accuracy.item()

def main():
    # Prepare data
    X_train, X_test, y_train, y_test = prepare_data()
    
    # Initialize model
    n_features = X_train.shape[1]
    model = BinaryClassifier(n_features)
    
    # Train model
    print("Training model...")
    train_model(model, X_train, y_train)
    
    # Evaluate model
    accuracy = evaluate_model(model, X_test, y_test)
    print(f'\nTest Accuracy: {accuracy:.4f}')

main()

Training model...
Epoch [10/100], Loss: 0.6247
Epoch [20/100], Loss: 0.4940
Epoch [30/100], Loss: 0.4156
Epoch [40/100], Loss: 0.3641
Epoch [50/100], Loss: 0.3277
Epoch [60/100], Loss: 0.3005
Epoch [70/100], Loss: 0.2794
Epoch [80/100], Loss: 0.2624
Epoch [90/100], Loss: 0.2483
Epoch [100/100], Loss: 0.2364

Test Accuracy: 0.9211
